# Instantiate compute environment

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.manifold import MDS
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from scipy import stats
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

# Read Data

In [2]:
df = pd.read_csv('data/processed/alles02.csv', header = 0)
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5).head()

,SMILES,InChI,EndPt,ReadyBiodeg
707,OS(=O)(=O)c1ccc(cc1)N(=O)=O,SPXOTSHWBDUUMT-UHFFFAOYSA-N,NRB,0
142,Cc1ccc(O)c(c1)n2n3n2c4ccccc34,YXJVVCASIQTSCQ-UHFFFAOYSA-N,NRB,0
407,CCCN(c1ccc(c(c1)NC(=O)C)/N=N/c1c(Br)cc(cc1C#N)...,MFZCNHQXBMFAEU-OCEACIFDSA-N,NRB,0
678,O=C1c2ccccc2c3cccc4cccc1c34,HUKPVYBUJRAUAG-UHFFFAOYSA-N,NRB,0
116,CC(=O)OCCN(CCOC(C)=O)c1ccc(N=Nc2sc3cc(Cl)ccc3n...,UGFKGKQLDQFUEG-UHFFFAOYSA-N,NRB,0


# Calculate features

In [3]:
nms = [x[0] for x in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
for i in range(len(df)):
    try:
        descrs = calc.CalcDescriptors(Chem.MolFromSmiles(df.iloc[i, 0]))
        for x in range(len(descrs)):
            df.at[i, str(nms[x])] = descrs[x]
    except:
        for x in range(len(descrs)):
            df.at[i, str(nms[x])] = 'NaN'   
            
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df = df.reset_index(drop=True)

In [4]:
df.sample(5).head()

,SMILES,InChI,EndPt,ReadyBiodeg,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
1209,Oc1ccc(cc1)C1(OC(=O)c2ccccc12)c1ccc(O)cc1,KJFMBFZCATUALV-UHFFFAOYSA-N,NRB,0,12.440656,-1.102106,12.440656,0.136310,0.709067,318.328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
605,CCOC(=O)CCCCC(=O)O,UZNLHJCCGYKCIL-UHFFFAOYSA-N,RB,1,10.718080,-0.820501,10.718080,0.124410,0.485018,174.196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1840,CN(C)CCN,DILRJUIACXKSQE-UHFFFAOYSA-N,NRB,0,5.187778,0.756944,5.187778,0.756944,0.493120,88.154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
712,Clc1ccccc1NNc2ccccc2Cl,IWKMQNKKYZERHI-UHFFFAOYSA-N,NRB,0,5.994774,0.654679,5.994774,0.654679,0.790286,253.132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
519,CCCCCCCCC=CCCCCCCCCC1CC(=O)OC1=O,UIQSPHZKORGIBU-UHFFFAOYSA-N,NRB,0,11.326629,-0.347641,11.326629,0.162702,0.137965,350.543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0


In [5]:
df.shape

(2043, 204)

# Apply property filters
Taken from:
* "Designing screens: how to make your hits a hit" W. Patrick Walters & Mark Namchuk, *Nature Reviews Drug Discovery*, **2**, 259 - 266 (2003)  
* "Streamlining lead discovery by aligning *in silico* and high-throughput screening" John W. Davies, Meir Glick, Jeremy L. Jenkins, *Current Opinion in Chemical Biology*, **10**(4), 343 - 351 (2006)

In [6]:
# molecular weight
dfFiltered = df[df['MolWt'] <= 700]
dfFiltered.shape

(2011, 204)

In [7]:
# heavy atom count
dfFiltered = dfFiltered[dfFiltered['HeavyAtomCount'] <= 50]
dfFiltered.shape

(2010, 204)

In [8]:
# number rotatable bonds
dfFiltered = dfFiltered[dfFiltered['NumRotatableBonds'] <= 12]
dfFiltered.shape

(1844, 204)

In [9]:
# H-bond donors
dfFiltered = dfFiltered[dfFiltered['NumHDonors'] <= 5]
dfFiltered.shape

(1831, 204)

In [10]:
# H-bond acceptors
dfFiltered = dfFiltered[dfFiltered['NumHAcceptors'] <= 10]
dfFiltered.shape

(1820, 204)

In [11]:
# hydrophobicity
dfFiltered = dfFiltered[dfFiltered['MolLogP'] <= 7.5]
dfFiltered.shape

(1802, 204)

In [12]:
# hydrophobicity
dfFiltered = dfFiltered[dfFiltered['MolLogP'] >= -5.0]
dfFiltered.shape

(1802, 204)

In [14]:
X = dfFiltered.drop(columns = ['SMILES', 'EndPt', 'InChI', 'ReadyBiodeg'])
y = np.ravel(dfFiltered[['ReadyBiodeg']])

# Feature Engineering

## Identify / remove near-zero variance descriptors

In [15]:
def variance_threshold_selector(data, threshold = 0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices = True)]]

nzv = variance_threshold_selector(X, 0.0)

X = X[nzv.columns]

In [16]:
print(X.shape)

(1802, 183)


## Identify / remove highly correlated descriptors

In [17]:
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
                                  k = 1).astype(np.bool))
to_drop = [column for column in upper.columns
           if any(upper[column] > 0.85)]

X = X[X.columns.drop(to_drop)]

In [18]:
print(X.shape)

(1802, 138)


In [23]:
list(X_train.columns);

## Standardize features by removing the mean and scaling to unit variance

In [19]:
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

## Nearest Neighbors

In [137]:
col_names =  ['expt', 'nn']
results = pd.DataFrame(columns = col_names)

# for i in range(len(X)):
    
#     # identify needle & haystack
#     X_haystack = np.delete(X, i, 0)
#     X_needle = np.take(X, i, 0)
#     y_haystack = np.delete(y, i, 0)
#     y_needle = np.take(y, i, 0)
    
#     # calculate distance from needle to every straw in the haystack
#     for j in range(len(X_haystack)):
#         distances.loc[j, 'dist'] = np.linalg.norm(X_needle - X_haystack[j])
        
#     euclid = distances.sort_values('dist').head(1)
    
#     results.loc[i, 'expt'] = y_needle
#     results.loc[i, 'nn'] = y_haystack[euclid.head().index.values[0]]
    
#     if (i % 5 == 0):
#         print(i)

# results

# dist = distance_matrix(X, X, p = 2, threshold = 1000000)
pd.DataFrame(np.delete(np.take(dist, 0, 0), 0, 0)).sort_values(1).head(1)

#     testAvgDist5.loc[i, 'dist'] = euclid[0]
#     if (i % 5 == 0):
#         print(i)

KeyError: 5

In [138]:
dist = distance_matrix(X, X, p = 2, threshold = 1000000)

In [153]:
pd.DataFrame(np.delete(np.take(dist, 5, 0), 5, 0)).sort_values(0).head(1)

,0
1278,0.0


In [157]:
print(np.take(dist, 2, 0))

[12.31990614 12.32361308  0.         ... 10.99668157 11.48242119
 11.12273005]


In [74]:
euclid.head().index.values

array([594], dtype=int64)

In [75]:
euclid.head().index.values[0]

594

In [79]:
y_haystack[euclid.head().index.values[0]]

0

In [96]:
from scipy.spatial import distance_matrix

In [108]:
dist = distance_matrix(X, X, p = 2, threshold = 1000000)

In [109]:
dist.shape

(1802, 1802)

In [ ]:
np.take(dist, 0, 0)

In [135]:
pd.DataFrame(np.delete(np.take(dist, 0, 0), 0, 0)).sort_values(0).head(1)

,0
594,6.055405
